# Taller Práctico 1: Construcción de Modelos Analíticos Orientados a la Segmentación de Clientes

**Objetivos del Taller:**

* Comprender la importancia de la segmentación de clientes en la estrategia de negocio.
* Aprender a preparar, analizar y segmentar datos de clientes.
* Desarrollar modelos de segmentación usando técnicas de clustering.

# Introducción a la Segmentación de Clientes

La segmentación de clientes consiste en dividir una base de clientes en grupos más pequeños y homogéneos según características, comportamientos o necesidades compartidas. Este proceso permite a las empresas comprender mejor a sus clientes, facilitando la toma de decisiones estratégicas en áreas clave como el desarrollo de productos, la gestión de operaciones, el servicio al cliente y, por supuesto, el marketing. A continuación, se detallan algunos beneficios clave de la segmentación de clientes:

* **Personalización de la Experiencia del Cliente:** La segmentación facilita la adaptación de la experiencia de cada cliente según sus preferencias y necesidades, lo que incrementa su satisfacción y fidelidad.

* **Optimización de Recursos y Operaciones:** Identificar los segmentos con mayores necesidades permite a la empresa priorizar recursos en áreas críticas, optimizando esfuerzos en distribución, servicio, soporte técnico, entre otros.

* **Desarrollo de Productos y Servicios:** Con un conocimiento más profundo de los diferentes perfiles de cliente, es posible crear productos y servicios adaptados a cada segmento, impulsando la innovación y atendiendo mejor las demandas específicas del mercado.

Esta estrategia no solo mejora la eficiencia operativa, sino que también fortalece las relaciones con los clientes, potenciando el crecimiento de la empresa en el largo plazo.

# Construcción de un Modelo Analítico Orientado a la Segmentación de Clientes

## Instalar las bibliotecas necesarias

Lo primero que debemos hacer es asegurarnos de tener todas las bibliotecas necesarias instaladas y cargadas en nuestro entorno de trabajo. Estas bibliotecas proporcionan las herramientas fundamentales para manipular datos, realizar análisis estadísticos, construir y evaluar modelos de machine learning. A continuación, se muestran las bibliotecas que utilizaremos a lo largo de este notebook:

* **numpy:** Biblioteca fundamental para operaciones matemáticas avanzadas y manipulación de arreglos en Python.

* **pandas:** Biblioteca poderosa para el análisis de datos, especialmente útil para trabajar con datos en formato de tablas (DataFrames).

* **seaborn:** Biblioteca para visualización de datos que facilita la creación de gráficos estadísticos claros y atractivos.

* **matplotlib:** Biblioteca esencial para crear visualizaciones y gráficos personalizados en Python.

* **sklearn:** Conjunto de herramientas de machine learning en Python que incluye algoritmos para clasificación, regresión y agrupamiento, entre otros.

In [ ]:
#!pip install numpy pandas seaborn matplotlib scikit-learn

## Cargar las bibliotecas previamente instaladas

Ahora que tenemos instaladas todas las bibliotecas necesarias, procedemos a cargarlas en nuestro entorno de trabajo.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

## Cargar el conjunto de datos

Con las bibliotecas listas, el siguiente paso es cargar nuestro conjunto de datos que utilizaremos para la construcción de nuestro modelo de segmentación de clientes.

In [ ]:
data = pd.read_csv("./data/customer_transactions_dataset_1.csv")

## Entendimiento de los Datos

Para construir un modelo de segmentación efectivo, es fundamental entender la estructura y las características del conjunto de datos. A continuación, exploraremos el conjunto de datos utilizando varias técnicas, incluyendo una inspección rápida de las primeras filas, un análisis general de las columnas, estadísticas descriptivas, y visualizaciones de las principales variables.

Comenzamos visualizando las primeras filas del conjunto de datos para obtener una idea inicial de las variables y su formato.

In [ ]:
# Muestra las primeras filas del conjunto de datos
data.head()

La función `info()` nos proporciona información general sobre el conjunto de datos, incluyendo el número de filas y columnas, los tipos de datos en cada columna y el número de valores no nulos, ayudándonos a detectar posibles datos faltantes o inconsistencias.

In [ ]:
# Muestra información general del conjunto de datos, como tipos de datos y valores nulos
data.info()

A través de la función `describe()` calculamos las estadísticas descriptivas de las variables numéricas, como la media, desviación estándar y valores extremos. Esto nos permite conocer la dispersión y centralización de los datos.

In [ ]:
data.describe()

Para analizar la dispersión y variabilidad de las compras, visualizamos la distribución de las variables `Cantidad` y `Valor`. La `Cantidad` muestra la variabilidad en la cantidad de productos comprados en cada transacción, mientras que `Valor` representa el monto total de cada transacción.

In [ ]:
# Configuración de subgráficos para visualización de la distribución de Cantidad y Valor
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Violinplot para la variable 'Cantidad' para una visualización más innovadora
sns.violinplot(data=data, x='Cantidad', ax=axes[0], inner="box", color="skyblue")
axes[0].set_title('Distribución de la Cantidad')
axes[0].set_xlabel('Cantidad')

# Histograma con estimación de densidad para la variable 'Valor'
sns.histplot(data=data, x='Valor', kde=True, ax=axes[1], color="salmon")
axes[1].set_title('Distribución del Valor de la Transacción')
axes[1].set_xlabel('Valor')

# Ajuste de espaciado entre subgráficos
plt.tight_layout()
plt.show()

A continuación, se presenta la distribución demográfica de los clientes en la base de datos según la variable "Segmento de Mercado" que en este contexto refleja la edad de los clientes.

In [ ]:
# Histograma con densidad para la variable 'Segmento de Mercado'
sns.histplot(data=data, x='Segmento de Mercado', kde=True, bins=15, color="skyblue")
plt.title('Distribución del Segmento de Mercado')
plt.xlabel('Edad')
plt.ylabel('Frecuencia')

# Mostrar la gráfica
plt.show()

Aquí visualizamos la cantidad de transacciones a través de cada canal de venta para identificar el uso de los diferentes canales por los clientes.

In [ ]:
# Gráfico de conteo para la variable 'Canal de Venta'
sns.countplot(x=data['Canal de Venta'])
plt.title('Distribución del Canal de Venta')
plt.show()

En esta celda, analizamos la evolución del valor total de las transacciones a lo largo del tiempo para identificar tendencias y patrones en las compras de los clientes.

In [ ]:
# Convertir la columna 'Fecha de Compra' a formato de fecha
data['Fecha de Compra'] = pd.to_datetime(data['Fecha de Compra'], errors='coerce')

# Agrupar datos por fecha para obtener el valor total de transacciones y el recuento de compras
data_by_date = data.groupby('Fecha de Compra').agg({'Valor': 'sum', 'Cantidad': 'count'}).reset_index()

# Gráfico de línea para visualizar la evolución del valor total de transacciones
plt.figure(figsize=(12, 6))
plt.plot(data_by_date['Fecha de Compra'], data_by_date['Valor'], marker='o')
plt.title('Evolución del Valor de Transacciones a lo Largo del Tiempo')
plt.xlabel('Fecha de Compra')
plt.ylabel('Valor Total de Transacciones')
plt.xticks(rotation=45)
plt.grid()
plt.show()

## Preparación de los Datos

Antes de realizar el análisis de segmentación, es esencial preparar el conjunto de datos. Esto implica limpiar y transformar los datos para que sean coherentes y adecuados para los algoritmos de machine learning. A continuación, realizamos varios pasos de preparación, incluyendo la eliminación de duplicados, el manejo de valores nulos, el cálculo de métricas relevantes por cliente, la selección de variables relevantes y el escalado de las mismas.

### Eliminación de registros duplicados

El primer paso que haremos es eliminar cualquier duplicado en el conjunto de datos, en este contexto quiere decir, eliminar aquellas transacciones que se registraron más de una vez por error. Los duplicados pueden afectar la precisión del modelo y distorsionar las métricas calculadas, como las frecuencias y promedios.

In [ ]:
# Eliminar registros duplicados en el conjunto de datos
data = data.drop_duplicates()
data.head()

### Eliminación de registros con valores nulos

A continuación, eliminamos las filas con valores nulos. Esto asegura que las métricas calculadas no se vean afectadas por datos incompletos. Es importante evaluar previamente si la eliminación de estas filas afecta significativamente el tamaño de la muestra.

In [ ]:
# Eliminar filas con valores nulos en el conjunto de datos
data = data.dropna()
data.head()

### Agrupación de datos por cliente

Como nuestro objetivo es construir un modelo de segmentación de clientes, debemos agrupar nuestro conjunto de datos a nivel de cliente (actualmente se encuentra a nivel de transacción, en donde un mismo cliente puede tener múltiples compras a lo largo del tiempo registrado), por esta razón, en esta celda, agrupamos el conjunto de datos a nivel de cliente y calculamos métricas que caracterizan el comportamiento del cliente a lo largo del histórico de sus transacciones, como la diversidad de productos, la cantidad promedio comprada, el valor promedio de las transacciones, la frecuencia de compra, etc.

In [ ]:
# Agrupación por cliente y cálculo de métricas relevantes
clientes_df = data.groupby(['ID Cliente', 'Nombre del Cliente']).agg({
    'Segmento de Mercado': 'first',           # Segmento de Mercado del Cliente
    'ID Producto': 'nunique',                 # Número de productos únicos adquiridos (Diversidad de Productos)
    'Cantidad': 'mean',                       # Cantidad promedio de productos comprados por transacción
    'Valor': 'mean',                          # Valor promedio de las compras por transacción
    'ID Cliente': 'count'                     # Número de compras (Frecuencia de compra)
}).rename(columns={
    'Segmento de Mercado': 'edad',
    'ID Producto': 'diversidad_de_productos',
    'Cantidad': 'promedio_de_cantidad',
    'Valor': 'promedio_de_valor',
    'ID Cliente': 'frecuencia_de_compra'
}).reset_index()

clientes_df.head()

### Encoding de variables categóricas

La mayoría de los algoritmos de machine learning requieren que todas las variables de entrada estén en una representación numérica. Esto significa que las variables categóricas, como el canal de venta, deben convertirse a una representación numérica para que el algoritmo de clustering pueda identificar patrones en los datos.

En este caso, utilizaremos el encoding de frecuencia categórica para la variable `Canal de Venta`. Este método cuenta cuántas veces cada cliente utiliza cada canal y crea una columna para cada canal con el número de transacciones. Así, transformamos la preferencia de canal de cada cliente de un formato categórico a un formato numérico que el modelo puede interpretar, facilitando la identificación de patrones de comportamiento.

In [ ]:
# Encoding de frecuencia categórica
clientes_df = data.groupby(['ID Cliente', 'Canal de Venta']).size().unstack(fill_value=0).reset_index().merge(clientes_df, on='ID Cliente', how='left')
clientes_df.head()

### Selección de variables relevantes

Ahora, eliminamos las columnas de identificación del cliente (ID Cliente y Nombre del Cliente). Esto se hace porque estas variables no aportan información relevante para el clustering y, al ser datos de identificación únicos, pueden introducir ruido en el modelo.

In [ ]:
clientes_df = clientes_df.drop(["ID Cliente", "Nombre del Cliente"], axis=1)
clientes_df.head()

### Normalización de las variables

Por último, debemos normalizar las variables, es decir, ajustar sus valores para que todas tengan la misma escala, independientemente de sus unidades originales. Esto es fundamental para algoritmos como K-Means, que utilizan distancias para formar grupos, ya que variables con rangos más amplios podrían dominar el análisis y sesgar los resultados. La normalización asegura que todas las variables contribuyan de manera equitativa al modelo.

En este caso, utilizamos StandardScaler, una técnica que transforma los datos para que cada variable tenga una media de 0 y una desviación estándar de 1. Esto garantiza que todas las variables numéricas estén en un rango comparable, independientemente de sus magnitudes originales.

Además, se creó una copia del DataFrame original antes de normalizar los datos. Esto es útil para analizar los datos originales (sin escalar) una vez finalizado el proceso de clustering, ya que nos permitirá interpretar las características de los clusters asignados en términos de las variables en su escala original.

In [ ]:
# Crear una copia de clientes_df antes de escalar las variables
clientes_df_original = clientes_df.copy()

# Crear un objeto escalador y aplicar el escalado a todas las columnas numéricas en clientes_df
scaler = StandardScaler()
clientes_df[list(clientes_df.columns)] = scaler.fit_transform(clientes_df[list(clientes_df.columns)])

# Visualizar las primeras filas del DataFrame final
clientes_df.head()

## Construcción del Modelo de Segmentación utilizando K-Means

Para este caso utilizaremos el algoritmo K-Means para segmentar a los clientes. K-Means es un algoritmo de aprendizaje no supervisado que se utiliza para agrupar datos en un número predefinido de clusters. Funciona asignando cada punto de datos al cluster cuyo centroide (la media de todos los puntos en ese cluster) es el más cercano. El proceso sigue estos pasos:

1. **Inicialización:** Selecciona al azar 'K' puntos del conjunto de datos como centroides iniciales.

2. **Asignación:** Asigna cada punto de datos al centroide más cercano, basándose en la distancia euclidiana.

3. **Actualización:** Recalcula los centroides como la media de todos los puntos asignados a ese cluster.

4. **Iteración:** Repite los pasos de asignación y actualización hasta que los centroides no cambien significativamente entre iteraciones, indicando que el modelo ha convergido.

### Búsqueda del hiperparámetro K óptimo

Determinar el número adecuado de clusters (`K`) es crucial para la eficacia del modelo K-Means. Hay varias técnicas para estimar el valor óptimo de este hiperparámetro:


#### Método del Codo

Este método implica graficar los resultados de varios modelos K-Means, cada uno con un número diferente de clusters, y buscar un punto donde el aumento en el número de clusters no resulte en una mejora significativa de la varianza explicada. Esto se visualiza típicamente como un punto de "codo" en el gráfico, donde la curva tiene una flexión pronunciada. El eje `x` representa el número de clusters y el eje `y` el valor de la suma de los cuadrados de las distancias (inercia) de cada punto a su centroide asignado.

$$
\text{Inercia} = \sum_{i=1}^{n} \min_{\mu_j \in C} \left( \| x_i - \mu_j \|^2 \right)
$$

In [ ]:
# Método del codo
distortions = []
K = range(2, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(clientes_df)
    distortions.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(K, distortions, 'bo-')
plt.xlabel('Número de Clusters')
plt.ylabel('Inercia')
plt.title('Método del Codo para Seleccionar Número de Clusters')
plt.show()

En el gráfico del Método del Codo, observamos que la curva tiene una disminución pronunciada de la inercia al aumentar el número de clusters de 2 a 4 y luego se estabiliza después de 4 clusters. Esto sugiere que incrementar el número de clusters más allá de 4 no ofrece una mejora significativa en la minimización de la inercia, lo que implica que 4 podría ser una buena elección para el número de clusters.

#### Método de la Silueta

Este método evalúa la calidad de los clusters calculando el score de silueta, que mide cuán similar es un punto a los puntos en su propio cluster en comparación con puntos en otros clusters. El score de silueta oscila entre -1 y 1, donde un valor alto indica que los puntos están bien agrupados y distantes de otros clusters. Este método no solo ayuda a determinar el número óptimo de clusters, sino también a evaluar la cohesión y separación de los clusters formados.

$$
s(i) = \frac{b(i) - a(i)}{\max \{ a(i), b(i) \}}
$$

In [ ]:
silhouette_scores = []
K = range(2, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(clientes_df)
    score = silhouette_score(clientes_df, kmeans.labels_)
    silhouette_scores.append(score)

plt.figure(figsize=(10, 6))
plt.plot(K, silhouette_scores, 'bo-')
plt.xlabel('Número de Clusters')
plt.ylabel('Puntuación de Silueta')
plt.title('Método de la Silueta para Seleccionar Número de Clusters')
plt.show()

Observando el gráfico, el score de silueta alcanza su máximo en 4 clusters y disminuye progresivamente con más clusters. Esto indica que con 4 clusters, los datos están más adecuadamente segmentados en términos de similitud interna y separación externa en comparación con otros números de clusters.

De esta manera, los resultados de ambos métodos, el Método del Codo y el Método de la Silueta, sugieren que 4 clusters son óptimos para este conjunto de datos $(k=4)$.

## Construcción del Modelo de Clustering (K-Means)

Una vez seleccionado el número óptimo de clusters, se construye el modelo final utilizando K-Means. A continuación, se configura y aplica el modelo K-Means al conjunto de datos preparado, utilizando 4 como el número de clusters, que se determinó como óptimo. Cada cliente en el dataframe original se etiqueta con un cluster, lo cual permite un análisis posterior detallado de las características y comportamientos de cada segmento.

In [ ]:
# Aplicación de KMeans con el número óptimo de clusters determinado previamente
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit_predict(clientes_df)

# Agregar las etiquetas de cluster al dataframe original para análisis posterior
clientes_df_original['Cluster'] = kmeans.labels_
clientes_df_original.head()

## Análisis de los resultados obtenidos

Tras aplicar el modelo K-Means y asignar cada cliente a uno de los cuatro clusters, analizamos las características principales de cada grupo. La siguiente serie de visualizaciones nos permitirá entender cómo se diferencian estos clusters en términos de comportamiento de compra, uso de canales de venta, y otros aspectos relevantes.

### Distribución de Densidad de Edad por Cluster

Para analizar cómo se distribuyen las edades dentro de cada cluster, utilizaremos un gráfico KDE (Estimación de Densidad Kernel). Este tipo de visualización es particularmente útil para entender la distribución subyacente de una variable continua como la edad.

In [ ]:
# Dibujar el KDE plot
plt.figure(figsize=(10, 6))
kde = sns.kdeplot(data=clientes_df_original, x='edad', hue='Cluster', fill=True, common_norm=False)
plt.title('Distribución de Densidad de Edad por Cluster')
plt.xlabel('Edad')
plt.ylabel('Densidad')
plt.show()

La gráfica anterior permite visualizar la distribución de las edades de los clientes en cada uno de los clusters definidos. En el Cluster 1, se identifica un segmento de clientes jóvenes, cuyas edades se concentran principalmente alrededor de los 20 años. Por otro lado, el Cluster 3 engloba un rango de edad más amplio que varía de los 20 a los 50 años, con una mayor presencia de edades intermedias, especialmente los 30 y 40 años. Asimismo, el Cluster 0 se caracteriza por incluir clientes de edades comprendidas entre los 40 y 70 años. Finalmente, el Cluster 2 representa a un grupo de clientes de mayor edad, probablemente jubilados o cercanos a la jubilación, con edades que oscilan entre los 55 y 70 años, destacando especialmente individuos de 60 años.

### Distribución de Cantidad Promedio de Productos por Cluster

Esta visualización compara la cantidad promedio de productos comprados en cada transacción entre los diferentes clusters.

In [ ]:
# Gráfico de caja para la cantidad promedio de productos comprados por transacción en cada cluster
plt.figure(figsize=(10, 6))
sns.boxplot(data=clientes_df_original, x='Cluster', y='promedio_de_cantidad')
plt.title('Distribución de Cantidad Promedio de Productos Comprados por Cluster')
plt.ylabel("Cantidad promedio de productos")
plt.show()

En la gráfica anterior, se observa la cantidad promedio de productos comprados por los clientes en cada grupo. El Cluster 0 muestra una compra moderada y uniforme de 5 productos, mientras que el Cluster 1, con la menor cantidad promedio, tiene una mediana de solo 2 productos, indicando compras enfocadas. En contraste, el Cluster 2 destaca por su alta cantidad promedio de productos comprados por transacción con una mediana de 12.6 productos por compra con una variabilidad considerable. Finalmente, el Cluster 3 presenta una mediana de 7 productos, colocándose como el segundo Cluster con mayor cantidad de productos comprados por transacción.

### Distribución de Valor Promedio por Compra por Cluster

Esta visualización compara la cantidad promedio de productos comprados en cada transacción entre los diferentes clusters.

In [ ]:
# Gráfico de caja para el valor total de las compras por transacción en cada cluster
plt.figure(figsize=(10, 6))
sns.boxplot(data=clientes_df_original, x='Cluster', y='promedio_de_valor')
plt.title('Distribución de Valor Promedio por Compra por Cluster')
plt.ylabel("Valor Promedio")
plt.show()

Esta gráfica ilustra el gasto promedio en las compras de cada cluster. El Cluster 0 muestra un gasto moderado, con una mediana de alrededor de 1.500 USD, y aunque la mayoría de las compras son de valores más bajos, algunos valores atípicos indican compras ocasionalmente más altas. Por otro lado, el Cluster 1 se caracteriza por compras de bajo valor, con una mediana cerca de 200 USD y pocos valores atípicos. En contraste, el Cluster 2 exhibe un gasto significativamente más alto por compra, con una mediana de 8.000 USD y valores que alcanzan más de $14,000, lo que lo identifica como el segmento con clientes de mayor valor, probablemente adquiriendo productos de alto precio. Finalmente, el Cluster 3, aunque con un gasto alto de una mediana de 4.000 USD, presenta variabilidad con varios outliers hacia abajo.

### Relación entre Cantidad Promedio y Valor Promedio por Cluster

Este gráfico permite visualizar la relación entre la cantidad promedio de productos comprados y el valor promedio de esas compras para cada cluster.

In [ ]:
# Crear el gráfico de dispersión
plt.figure(figsize=(10, 6))
sns.scatterplot(data=clientes_df_original, x='promedio_de_cantidad', y='promedio_de_valor', hue='Cluster', style='Cluster', s=100, palette='viridis')

# Añadir títulos y etiquetas
plt.title('Relación entre Cantidad Promedio y Valor Promedio por Cluster')
plt.xlabel('Promedio de Cantidad')
plt.ylabel('Promedio de Valor')
plt.legend(title='Cluster')

# Mostrar el gráfico
plt.show()

La gráfica anterior nos permite tener una visión integral de cómo la cantidad de productos comprados se relaciona con el valor promedio gastado en cada compra, distribuidos entre los diferentes clusters.

### Distribución de Frecuencia de Compra por Cluster

Esta visualización utiliza un histograma apilado para comparar la frecuencia de compra entre los diferentes clusters. Nos permite ver claramente cómo se distribuye la frecuencia de las compras entre los grupos y es útil para entender qué tan a menudo los clientes de cada cluster compran.

In [ ]:
# Histograma apilado para la frecuencia de compras por cluster
plt.figure(figsize=(10, 6))
sns.histplot(data=clientes_df_original, x='frecuencia_de_compra', hue='Cluster', multiple='stack', bins=10)
plt.title('Distribución de Frecuencia de Compra por Cluster')
plt.show()

La gráfica anterior muestra cómo varía la cantidad de clientes por cluster a lo largo de diferentes frecuencias de compra. El Cluster 2 se destaca en los rangos más bajos, indicando compras esporádicas que podrían beneficiarse de incentivos para aumentar la frecuencia. El Cluster 0 tiene una presencia uniforme desde frecuencias bajas a intermedias, sugiriendo una variabilidad en el comportamiento de compra de sus clientes. En contraste, el Cluster 1 es predominante en las frecuencias intermedias y altas, mostrando un patrón de compra regular, mientras que el Cluster 3 sobresale en las frecuencias más altas, identificándose como un grupo de compradores habituales y fieles, ideales para estrategias de retención y aumento de valor a largo plazo.

### Heatmap de Promedio de Uso de Canales de Venta por Cluster

Este heatmap proporciona una visión general del uso medio de los diferentes canales de venta por cluster. Los colores más cálidos indican un uso más frecuente.

In [ ]:
# Calcula el uso promedio de canales de venta por cluster y crea un heatmap
channel_usage_summary = clientes_df_original.groupby('Cluster')[['Distribuidor', 'En Línea', 'Telefónica', 'Tienda Física']].mean().round(2)

plt.figure(figsize=(8, 6))
sns.heatmap(channel_usage_summary, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Promedio de Compras'})
plt.title('Heatmap de Promedio de Uso de Canales de Venta por Cluster')
plt.xlabel('Canal de Venta')
plt.ylabel('Cluster')
plt.show()

Esta gráfica nos ofrece una visión clara de las preferencias de canales de venta de diferentes grupos de clientes. En el Cluster 0, hay un predominio en el uso de Tienda Física y Telefónica. El Cluster 1 muestra una fuerte preferencia por el canal En Línea, con un uso significativamente menor de Tienda Física y Telefónica, destacando la valoración de la autonomía en el proceso de compra. Por otro lado, el Cluster 2 prefiere el canal Distribuidor, sugiriendo una inclinación hacia compras a través de intermediarios, posiblemente en formatos de venta al por mayor. Finalmente, el Cluster 3 combina una fuerte preferencia por Tienda Física y En Línea, evidenciando un comportamiento de compra híbrido con un uso moderado de Telefónica y Distribuidor, lo que indica una diversidad en los métodos de compra con una clara inclinación hacia la interacción directa antes de realizar una compra.

## Caracterización final de los Clusters Obtenidos

A partir de los resultados obtenidos anteriormente, podemos realizar un perfilamiento de los Clusters obtenidos.

### Cluster 0: Cliente Maduro y Moderado
* **Edad:** 40 a 70 años.
* **Comportamiento de compra:** Moderado con una mediana de 5 productos por transacción y un gasto medio de 1.500 USD.
* **Frecuencia de compra:** Variabilidad desde baja a intermedia.
* **Canal de venta preferido:** Tienda física y telefónica, lo que indica un enfoque en la interacción personal y la consulta directa antes de comprar.

### Cluster 1: Cliente Joven y Digital
* **Edad:** Concentración alrededor de los 20 años.
* **Comportamiento de compra:** Compras enfocadas con una mediana de 2 productos y gasto bajo de cerca de 200 USD.
* **Frecuencia de compra:** Alta, con un patrón de compra regular.
* **Canal de venta preferido:** En línea, destacando un enfoque hacia la autonomía y comodidad del comercio electrónico.

### Cluster 2: Cliente Sénior de Alto Valor
* **Edad:** 55 a 70 años, con un pico en 60 años.
* **Comportamiento de compra:** Alta cantidad de productos por compra (mediana de 12.6 productos) con el mayor gasto promedio por transacción (mediana de 8.000 USD).
* **Frecuencia de compra:** Baja, con compras esporádicas que podrían ser grandes adquisiciones.
* **Canal de venta preferido:** Distribuidor, indicando una preferencia por compras posiblemente al por mayor o de alto valor a través de intermediarios.

### Cluster 3: Cliente Adulto y Leal
* **Edad:** 20 a 50 años, con una mayor presencia en los 30 y 40 años.
* **Comportamiento de compra:** Buena cantidad de productos (mediana de 7 productos) con un gasto considerable (mediana de 4.000 USD).
* **Frecuencia de compra:** Alta, mostrando lealtad y habitualidad en sus compras.
* **Canal de venta preferido:** Combinación de tienda física y en línea, sugiriendo un comportamiento híbrido que valoriza tanto la experiencia en tienda como la comodidad del comercio electrónico.